In [75]:
import tensorflow as tf
tf.config.set_visible_devices([], device_type='GPU')

In [76]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from tqdm.auto import tqdm

from typing import Any, Callable, Sequence, Union
import numpy as np
import scipy.stats as stats

import tensorflow as tf
tf.config.set_visible_devices([], device_type='GPU')

import jax
from jax import lax, random, numpy as jnp
import flax
from flax.core import freeze, unfreeze, FrozenDict
from flax import linen as nn
from flax import struct
from flax.training import train_state
from flax.training import orbax_utils

import optax
import orbax.checkpoint

from clu import metrics
from ml_collections import ConfigDict

from einops import reduce, rearrange
import wandb

from iqadatasets.datasets import *
from fxlayers.layers import *
from fxlayers.layers import GaborLayerLogSigma_, GaussianLayerGamma, FreqGaussianGamma, OrientGaussianGamma
from fxlayers.initializers import *
from JaxPlayground.utils.constraints import *
from JaxPlayground.utils.wandb import *

In [77]:
# dst_train = TID2008("/lustre/ific.uv.es/ml/uv075/Databases/IQA//TID/TID2008/", exclude_imgs=[25])
# dst_val = TID2013("/lustre/ific.uv.es/ml/uv075/Databases/IQA//TID/TID2013/", exclude_imgs=[25])
# dst_train = TID2008("/media/disk/databases/BBDD_video_image/Image_Quality//TID/TID2008/", exclude_imgs=[25])
# dst_val = TID2013("/media/disk/databases/BBDD_video_image/Image_Quality//TID/TID2013/", exclude_imgs=[25])
# dst = KADIK10K("/media/disk/databases/BBDD_video_image/Image_Quality/KADIK10K/")
# dst = PIPAL("/media/disk/databases/BBDD_video_image/Image_Quality/PIPAL/")
dst = KADIK10K("/lustre/ific.uv.es/ml/uv075/Databases/IQA/KADIK10K/")
# dst = KADIK10K("/media/databases/IQA/KADIK10K/")

In [78]:
img, img_dist, mos = next(iter(dst.dataset))
img.shape, img_dist.shape, mos.shape

2024-04-29 10:21:37.621460: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [10125]
	 [[{{node Placeholder/_2}}]]


(TensorShape([384, 512, 3]), TensorShape([384, 512, 3]), TensorShape([]))

In [79]:
id = "pzy7mqaf" # MSE
# id = "gptw703y" # JS
# id = "d36wb0zf" # KLD

In [80]:
api = wandb.Api()
prev_run = api.run(f"jorgvt/PerceptNet_KLD/{id}")

In [81]:
config = ConfigDict(prev_run.config["_fields"])
config

BATCH_SIZE: 32
EPOCHS: 500
LEARNING_RATE: 0.0003
METRIC: MSE
SEED: 42

In [82]:
for file in prev_run.files():
    file.download(root=prev_run.dir, replace=True)

In [83]:
wandb.init(project="PerceptNet_KLD",
           name=prev_run.name,
           job_type="evaluate",
           mode="online",
           )
config = config
config

BATCH_SIZE: 32
EPOCHS: 500
LEARNING_RATE: 0.0003
METRIC: MSE
SEED: 42

In [84]:
def resize(img, img_dist, mos):
    h, w = 384, 512
    img = tf.image.resize(img, (h//8, w//8))
    img_dist = tf.image.resize(img_dist, (h//8, w//8))
    return img, img_dist, mos

In [85]:
dst_rdy = dst.dataset.batch(config.BATCH_SIZE, num_parallel_calls=tf.data.AUTOTUNE)\
                     .map(resize)

## Define the model we're going to use

> It's going to be a very simple model just for demonstration purposes.

In [86]:
class PerceptNet(nn.Module):
    """IQA model inspired by the visual system."""

    @nn.compact
    def __call__(self,
                 inputs,
                 **kwargs,
                 ):
        outputs = rearrange(inputs, "b h w c -> b (h w c)")
        if config.METRIC == "KLD" or config.METRIC == "JS":
            mean = nn.Dense(features=2)(outputs)
            std = nn.Dense(features=2)(outputs)
            return mean, std
        elif config.METRIC == "MSE":
            return nn.Dense(features=2)(outputs)

In [87]:
# class PerceptNet(nn.Module):
#     """IQA model inspired by the visual system."""

#     @nn.compact
#     def __call__(self,
#                  inputs,
#                  **kwargs,
#                  ):
#         outputs = rearrange(inputs, "b h w c -> b (h w c)")
#         mean = nn.Dense(features=2)(outputs)
#         std = nn.Dense(features=2)(outputs)
#         return mean, std

## Define the metrics with `clu`

In [88]:
@struct.dataclass
class Metrics(metrics.Collection):
    """Collection of metrics to be tracked during training."""
    loss: metrics.Average.from_output("loss")

By default, `TrainState` doesn't include metrics, but it's very easy to subclass it so that it does:

In [89]:
class TrainState(train_state.TrainState):
    metrics: Metrics
    state: FrozenDict

We'll define a function that initializes the `TrainState` from a module, a rng key and some optimizer:

In [90]:
def create_train_state(module, key, tx, input_shape):
    """Creates the initial `TrainState`."""
    variables = module.init(key, jnp.ones(input_shape))
    state, params = variables.pop('params')
    return TrainState.create(
        apply_fn=module.apply,
        params=params,
        state=state,
        tx=tx,
        metrics=Metrics.empty()
    )

## Define evaluation step

In [91]:
def kld(mean_p, std_p, mean_q, std_q, axis=(1)):
    """Assume diagonal covariance matrix and that the input is the logvariance."""
    std_p, std_q = jnp.exp(std_p), jnp.exp(std_q)
    def safe_div(a, b): return a/b #jnp.where(a == b, 1, a/b)
    det_p = jnp.prod(std_p, axis=axis) + 1e-5
    det_q = jnp.prod(std_q, axis=axis) + 1e-5
    
    return jnp.log(safe_div(det_p, det_q)) + jnp.sum((1/std_q)*(mean_p - mean_q)**2, axis=axis) + jnp.sum(std_p/std_q, axis=axis)

In [92]:
def js(mean_p, std_p, mean_q, std_q, axis=(1)):
    return (1/2)*(kld(mean_p, std_p, mean_q, std_q, axis) + kld(mean_q, std_q, mean_p, std_p, axis))

In [93]:
@jax.jit
def compute_distance(*, state, batch):
    """Obtaining the metrics for a given batch."""
    img, img_dist, mos = batch
    
    ## Calculate the KLD
    if config.METRIC == "KLD":
        (img_mean, img_std), updated_state = state.apply_fn({"params": state.params, **state.state}, img, mutable=list(state.state.keys()), train=False)
        (img_dist_mean, img_dist_std), updated_state = state.apply_fn({"params": state.params, **state.state}, img_dist, mutable=list(state.state.keys()), train=False)
        dist = kld(img_mean, img_std, img_dist_mean, img_dist_std)
    
    elif config.METRIC == "JS":
        (img_mean, img_std), updated_state = state.apply_fn({"params": state.params, **state.state}, img, mutable=list(state.state.keys()), train=False)
        (img_dist_mean, img_dist_std), updated_state = state.apply_fn({"params": state.params, **state.state}, img_dist, mutable=list(state.state.keys()), train=False)
        dist = js(img_mean, img_std, img_dist_mean, img_dist_std)
    
    elif config.METRIC == "MSE":
        img_pred, updated_state = state.apply_fn({"params": state.params, **state.state}, img, mutable=list(state.state.keys()), train=True)
        img_dist_pred, updated_state = state.apply_fn({"params": state.params, **state.state}, img_dist, mutable=list(state.state.keys()), train=True)
        ## Calculate the MSE
        dist = ((img_pred - img_dist_pred)**2).sum(axis=(1))**(1/2)
    
    return dist

## Load the pretrained model!

In [94]:
state = create_train_state(PerceptNet(), random.PRNGKey(config.SEED), optax.adam(config.LEARNING_RATE), input_shape=(1,384//8,512//8,3))

In [95]:
def check_trainable(path):
    return False
    # return ("A" in path) or ("alpha_achrom" in path) or ("alpha_chrom_rg" in path) or ("alpha_chrom_yb" in path)

In [96]:
trainable_tree = freeze(flax.traverse_util.path_aware_map(lambda path, v: "non_trainable" if check_trainable(path)  else "trainable", state.params))

In [97]:
param_count = sum(x.size for x in jax.tree_util.tree_leaves(state.params))
trainable_param_count = sum([w.size if t=="trainable" else 0 for w, t in zip(jax.tree_util.tree_leaves(state.params), jax.tree_util.tree_leaves(trainable_tree))])
param_count, trainable_param_count

(18434, 18434)

In [98]:
wandb.run.summary["total_parameters"] = param_count
wandb.run.summary["trainable_parameters"] = trainable_param_count

In [99]:
optimizers = {
    "trainable": optax.adam(learning_rate=config.LEARNING_RATE),
    "non_trainable": optax.set_to_zero(),
}

In [100]:
tx = optax.multi_transform(optimizers, trainable_tree)

In [101]:
state = create_train_state(PerceptNet(), random.PRNGKey(config.SEED), tx, input_shape=(1,384//8,512//8,3))

In [102]:
# Before actually training the model we're going to set up the checkpointer to be able to save our trained models:
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
save_args = orbax_utils.save_args_from_target(state)

In [103]:
# Load weights
state = orbax_checkpointer.restore(os.path.join(prev_run.dir,"model-best"), item=state)

## Evaluate!

In [104]:
metrics_history = {
    "distance": [],
    "mos": [],
}

In [105]:
%%time
for batch in tqdm(dst_rdy.as_numpy_iterator()):
    img, img_dist, mos = batch
    distance = compute_distance(state=state, batch=batch)
    metrics_history["distance"].extend(distance)
    metrics_history["mos"].extend(mos)
    # break

2024-04-29 10:21:51.165082: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [10125]
	 [[{{node Placeholder/_0}}]]
2024-04-29 10:21:51.166125: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [10125]
	 [[{{node Placeholder/_2}}]]


0it [00:00, ?it/s]

CPU times: user 4min 36s, sys: 42.1 s, total: 5min 18s
Wall time: 6.38 s


In [106]:
assert len(metrics_history["distance"]) == len(dst.data)

In [107]:
stats.pearsonr(metrics_history["distance"], metrics_history["mos"]), stats.spearmanr(metrics_history["distance"], metrics_history["mos"])

(PearsonRResult(statistic=-0.4060353975112158, pvalue=0.0),
 SignificanceResult(statistic=-0.5671970127107808, pvalue=0.0))

In [108]:
results = dst.data.copy()
results["Distance"] = metrics_history["distance"]
results.head()

,dist_img,ref_img,dmos,var,Distance
0,I01_01_01.png,I01.png,4.57,0.496,0.0
1,I01_01_02.png,I01.png,4.33,0.869,0.057725802
2,I01_01_03.png,I01.png,2.67,0.789,0.22373466
3,I01_01_04.png,I01.png,1.67,0.596,0.49756685
4,I01_01_05.png,I01.png,1.10,0.300,0.9510435


In [109]:
wandb.log({"KADID10K": wandb.Table(dataframe=results),
           "KADID10K_pearson": stats.pearsonr(metrics_history["distance"], metrics_history["mos"])[0],
           "KADID10K_spearman": stats.spearmanr(metrics_history["distance"], metrics_history["mos"])[0],
           })

In [110]:
wandb.finish()

KADID10K_pearson,▁
KADID10K_spearman,▁
KADID10K_pearson,-0.40604
KADID10K_spearman,-0.5672
total_parameters,18434
trainable_parameters,18434
